In [1]:

import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.pca import H2OPrincipalComponentAnalysisEstimator
from h2o.transforms.decomposition import H2OPCA
import tempfile
import os
from os import listdir
from os import path
from os.path import isfile, join, getsize
import pandas as pd
import numpy as np
from pycaret.classification import *
import seaborn as sns
import pickle
import tqdm


Bad key "text.kerning_factor" on line 4 in
/opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])



In [3]:
def compute_ap(recall, precision):
    """ Compute the average precision, given the recall and precision curves.
    Code originally from https://github.com/rbgirshick/py-faster-rcnn.
    # Arguments
        recall:    The recall curve (list).
        precision: The precision curve (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.0], recall, [1.0]))
    mpre = np.concatenate(([0.0], precision, [0.0]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

def ap_per_class(tp, conf, pred_cls, target_cls):
    """ Compute the average precision, given the recall and precision curves.
    Source: https://github.com/rafaelpadilla/Object-Detection-Metrics.
    # Arguments
        tp:    True positives (list).
        conf:  Objectness value from 0-1 (list).
        pred_cls: Predicted object classes (list).
        target_cls: True object classes (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """

    # Sort by objectness
    i = np.argsort(-conf)
    tp, conf, pred_cls = tp[i], conf[i], pred_cls[i]

    # Find unique classes
    unique_classes = np.unique(target_cls)

    # Create Precision-Recall curve and compute AP for each class
    ap, p, r = [], [], []
    for c in tqdm.tqdm(unique_classes, desc="Computing AP"):
        i = pred_cls == c
        n_gt = (target_cls == c).sum()  # Number of ground truth objects
        n_p = i.sum()  # Number of predicted objects

        if n_p == 0 and n_gt == 0:
            continue
        elif n_p == 0 or n_gt == 0:
            ap.append(0)
            r.append(0)
            p.append(0)
        else:
            # Accumulate FPs and TPs
            fpc = (1 - tp[i]).cumsum()
            tpc = (tp[i]).cumsum()

            # Recall
            recall_curve = tpc / (n_gt + 1e-16)
            r.append(recall_curve[-1])

            # Precision
            precision_curve = tpc / (tpc + fpc)
            p.append(precision_curve[-1])

            # AP from recall-precision curve
            ap.append(compute_ap(recall_curve, precision_curve))

    # Compute F1 score (harmonic mean of precision and recall)
    p, r, ap = np.array(p), np.array(r), np.array(ap)
    f1 = 2 * p * r / (p + r + 1e-16)

    return p, r, ap, f1, unique_classes.astype("int32")

In [4]:
h2o.init(max_mem_size="64G", nthreads = 30)
#nthreads = 30

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.7" 2020-04-14; OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)
  Starting server from /opt/tljh/user/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpsxu2wbg_
  JVM stdout: /tmp/tmpsxu2wbg_/h2o_jupyter_chandler_vaughn_started_from_python.out
  JVM stderr: /tmp/tmpsxu2wbg_/h2o_jupyter_chandler_vaughn_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.3
H2O_cluster_version_age:,1 month and 6 days
H2O_cluster_name:,H2O_from_python_jupyter_chandler_vaughn_fwvlq1
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,64 Gb
H2O_cluster_total_cores:,32
H2O_cluster_allowed_cores:,30
H2O_cluster_status:,"accepting new members, healthy"


In [5]:
os.chdir('/home/jupyter-chandler.vaughn/data/')
infile = open('merged_training.pickle','rb')
merge_pairs_plus_random = pickle.load(infile)
infile.close()
merge_pairs_plus_random = merge_pairs_plus_random.drop(columns=['pair', 'song_1', 'song_2', 'pair_index', 'sim_distance', 'sim_distance_argmin'])
merge_pairs_plus_random.reset_index(drop=True, inplace=True)

from sklearn.decomposition import PCA
x = merge_pairs_plus_random.drop(['target'], axis=1)
pca = PCA(n_components=4)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['PC1', 'PC2', 'PC3', 'PC4'])
finalDf = pd.concat([principalDf, merge_pairs_plus_random[['target']]], axis = 1)

# create an export checkpoints directory
checkpoints_dir = tempfile.mkdtemp()

dataset = h2o.H2OFrame(finalDf)
#dataset = h2o.import_file('final_dataset_new.csv')
dataset.shape

Parse progress: |█████████████████████████████████████████████████████████| 100%


(314, 5)

In [6]:
# Identify predictors and response
x = dataset.columns
y = "target"
x.remove(y)

#dataset[y].unique()
dataset[y] = dataset[y].asfactor()

In [7]:
#impute any na's to zero
dataset[x].impute(column = 0, values = [0 for c in range(dataset[x].ncol)])
dataset[x].isna().any()

False

In [8]:
#subset the data so that we can work with it
#eventually we need to do all data for modeling
#using splitframe so we get random rows
#subset_dataset_train, subset_dataset_test, therest = dataset.split_frame(ratios=[0.04, 0.04])

In [9]:
print("Rows To Process: " + str(dataset.nrows))
print("Dataframe Shape: " + str(dataset.shape))

Rows To Process: 314
Dataframe Shape: (314, 5)


In [10]:
#subset the data so that we can work with it
#eventually we need to do all data for modeling
#subset_dataset = dataset[range(0,2000,1),:]
dataset[y] = dataset[y].asfactor()
dataset[x] = dataset[x].asnumeric() 
train,test,valid = dataset.split_frame(ratios=[.7, .15])

In [ ]:
aml = H2OAutoML(max_models=100, seed=1, balance_classes = True, exclude_algos = ["GBM"], stopping_metric='logloss')
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████

In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows) 

In [62]:
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20200617_205147

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.04779401403316598
RMSE: 0.218618421074634
LogLoss: 0.2176163159468597
Null degrees of freedom: 225
Residual degrees of freedom: 211
Null deviance: 241.36051425996396
Residual deviance: 98.36257480798056
AIC: 128.36257480798056
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6079090491563213: 


,,0,1,Error,Rate
0,0,175.0,0.0,0.0,(0.0/175.0)
1,1,0.0,51.0,0.0,(0.0/51.0)
2,Total,175.0,51.0,0.0,(0.0/226.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.607909,1.000000,50.0
1,max f2,0.607909,1.000000,50.0
2,max f0point5,0.607909,1.000000,50.0
3,max accuracy,0.607909,1.000000,50.0
4,max precision,0.887533,1.000000,0.0
5,max recall,0.607909,1.000000,50.0
6,max specificity,0.887533,1.000000,0.0
7,max absolute_mcc,0.607909,1.000000,50.0
8,max min_per_class_accuracy,0.607909,1.000000,50.0
9,max mean_per_class_accuracy,0.607909,1.000000,50.0



Gains/Lift Table: Avg response rate: 22.57 %, avg score: 33.44 %


...

,,0,1,Error,Rate
0,0,173.0,2.0,0.0114,(2.0/175.0)
1,1,22.0,29.0,0.4314,(22.0/51.0)
2,Total,195.0,31.0,0.1062,(24.0/226.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.569932,0.707317,30.0
1,max f2,0.150522,0.740741,92.0
2,max f0point5,0.596214,0.838323,28.0
3,max accuracy,0.596214,0.893805,28.0
4,max precision,0.880579,1.000000,0.0
5,max recall,0.057434,1.000000,216.0
6,max specificity,0.880579,1.000000,0.0
7,max absolute_mcc,0.596214,0.679072,28.0
8,max min_per_class_accuracy,0.188679,0.803922,74.0
9,max mean_per_class_accuracy,0.222283,0.806835,60.0



Gains/Lift Table: Avg response rate: 22.57 %, avg score: 22.55 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.013274,0.853419,4.431373,4.431373,1.000000,0.866606,1.000000,0.866606,0.058824,0.058824,343.137255,343.137255
1,,2,0.022124,0.849701,4.431373,4.431373,1.000000,0.851783,1.000000,0.860677,0.039216,0.098039,343.137255,343.137255
2,,3,0.030973,0.845429,4.431373,4.431373,1.000000,0.847735,1.000000,0.856979,0.039216,0.137255,343.137255,343.137255
3,,4,0.044248,0.839207,4.431373,4.431373,1.000000,0.842516,1.000000,0.852640,0.058824,0.196078,343.137255,343.137255
4,,5,0.053097,0.820500,4.431373,4.431373,1.000000,0.826042,1.000000,0.848207,0.039216,0.235294,343.137255,343.137255
5,,6,0.101770,0.667366,4.028520,4.238704,0.909091,0.759782,0.956522,0.805917,0.196078,0.431373,302.852050,323.870418
6,,7,0.150442,0.457066,2.819964,3.779700,0.636364,0.582977,0.852941,0.733789,0.137255,0.568627,181.996435,277.970012
7,,8,0.203540,0.303310,1.107843,3.082694,0.250000,0.369251,0.695652,0.638692,0.058824,0.627451,10.784314,208.269395
8,,9,0.300885,0.201542,1.409982,2.541522,0.318182,0.244555,0.573529,0.511177,0.137255,0.764706,40.998217,154.152249
9,,10,0.402655,0.153093,0.770673,2.093945,0.173913,0.178581,0.472527,0.427115,0.078431,0.843137,-22.932651,109.394527


In [73]:
pred = aml.predict(test)
pred

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict,p0,p1
1,0.115595,0.884405
1,0.256495,0.743505
1,0.612095,0.387905
0,0.723899,0.276101
1,0.114197,0.885803
0,0.704621,0.295379
0,0.692917,0.307083
0,0.885767,0.114233
0,0.93987,0.0601304
0,0.686095,0.313905


In [ ]:
results = pd.concat([test.as_data_frame(), pred.as_data_frame()], axis=1)
results['rank'] = results['p1'].rank(ascending=False)

In [89]:
#results_sorted = results.sort_values(by=['p1'], ascending=False)
results_sorted = results.sort_values(by=['rank'], ascending=True)
results_sorted

,sim_distance_bass,sim_distance_drums,sim_distance_vocals,sim_distance_other,target,sim_distance_argmin_combined,predict,p0,p1,rank
4,-2.920422,-3.412313,-3.394648,-3.616834,1,-3.754379,1,0.114197,0.885803,1.0
0,-4.010184,-3.278129,-3.577138,-3.629846,1,-4.010184,1,0.115595,0.884405,2.0
1,-2.809863,-2.572479,-3.097211,-3.313611,1,-3.708466,1,0.256495,0.743505,3.0
25,-3.132435,-3.431346,-2.905503,-3.347269,0,-3.431346,1,0.289677,0.710323,4.0
20,-3.039177,-3.506325,-2.923861,-2.918168,0,-3.506325,1,0.517035,0.482965,5.0
13,-2.297682,-2.946006,-2.799378,-2.922557,0,-2.946006,1,0.574292,0.425708,6.0
19,-3.219290,-3.058765,-3.033054,-3.193511,0,-3.357223,1,0.591405,0.408595,7.0
2,-2.716459,-3.243444,-2.800118,-2.961710,1,-3.243444,1,0.612095,0.387905,8.0
23,-2.594694,-2.818501,-2.877898,-3.102851,0,-3.129494,0,0.627291,0.372709,9.0
12,-3.168218,-3.175548,-3.098829,-3.125633,0,-3.205519,0,0.647546,0.352454,10.0


In [118]:
#find Average Precision
tp = results_sorted['target'].values
conf = results_sorted['p1'].values
pred_cls = results_sorted['predict'].values
target_cls = [results_sorted['target'].values]

p, r, ap, f1, unique_classes = ap_per_class(tp, conf, pred_cls, target_cls)

Computing AP: 100%|██████████| 2/2 [00:00<00:00, 2257.43it/s]


In [120]:
#MAP
np.mean(ap)

0.2685185185185185

In [122]:
perf = aml.leader.model_performance(valid)
perf


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.12625186361044965
RMSE: 0.35531938254259315
LogLoss: 0.4229858515972594
Null degrees of freedom: 53
Residual degrees of freedom: 39
Null deviance: 74.47338770323641
Residual deviance: 45.68247197250401
AIC: 75.682471972504
AUC: 0.8451127819548871
AUCPR: 0.850552428865484
Gini: 0.6902255639097743

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5728661888878201: 


,,0,1,Error,Rate
0,0,35.0,0.0,0.0,(0.0/35.0)
1,1,6.0,13.0,0.3158,(6.0/19.0)
2,Total,41.0,13.0,0.1111,(6.0/54.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.572866,0.812500,12.0
1,max f2,0.093488,0.775862,39.0
2,max f0point5,0.572866,0.915493,12.0
3,max accuracy,0.572866,0.888889,12.0
4,max precision,0.883328,1.000000,0.0
5,max recall,0.053596,1.000000,53.0
6,max specificity,0.883328,1.000000,0.0
7,max absolute_mcc,0.572866,0.764253,12.0
8,max min_per_class_accuracy,0.403015,0.736842,17.0
9,max mean_per_class_accuracy,0.572866,0.842105,12.0



Gains/Lift Table: Avg response rate: 35.19 %, avg score: 34.64 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.018519,0.882194,2.842105,2.842105,1.000000,0.883328,1.000000,0.883328,0.052632,0.052632,184.210526,184.210526
1,,2,0.037037,0.880758,2.842105,2.842105,1.000000,0.881188,1.000000,0.882258,0.052632,0.105263,184.210526,184.210526
2,,3,0.037037,0.876963,0.000000,2.842105,0.000000,0.000000,1.000000,0.882258,0.000000,0.105263,-100.000000,184.210526
3,,4,0.055556,0.872894,2.842105,2.842105,1.000000,0.874027,1.000000,0.879514,0.052632,0.157895,184.210526,184.210526
4,,5,0.055556,0.867890,0.000000,2.842105,0.000000,0.000000,1.000000,0.879514,0.000000,0.157895,-100.000000,184.210526
5,,6,0.111111,0.843343,2.842105,2.842105,1.000000,0.853855,1.000000,0.866684,0.157895,0.315789,184.210526,184.210526
6,,7,0.148148,0.808650,2.842105,2.842105,1.000000,0.838352,1.000000,0.859601,0.105263,0.421053,184.210526,184.210526
7,,8,0.203704,0.599264,2.842105,2.842105,1.000000,0.723236,1.000000,0.822411,0.157895,0.578947,184.210526,184.210526
8,,9,0.296296,0.416359,1.136842,2.309211,0.400000,0.524863,0.812500,0.729427,0.105263,0.684211,13.684211,130.921053
9,,10,0.407407,0.331501,0.473684,1.808612,0.166667,0.361663,0.636364,0.629128,0.052632,0.736842,-52.631579,80.861244


In [129]:
#Lets look at Plagerized data
plagerize_unseen = pd.read_pickle('merged_unseen.pickle')

In [130]:
plagerize_unseen['sim_distance_bass'] = np.log(plagerize_unseen['sim_distance_bass']+0.0001)
plagerize_unseen['sim_distance_drums'] = np.log(plagerize_unseen['sim_distance_drums']+0.0001)
plagerize_unseen['sim_distance_vocals'] = np.log(plagerize_unseen['sim_distance_vocals']+0.0001)
plagerize_unseen['sim_distance_other'] = np.log(plagerize_unseen['sim_distance_other']+0.0001)
plagerize_unseen['sim_distance_argmin_combined'] = plagerize_unseen[['sim_distance','sim_distance_argmin']].min(axis=1)
plagerize_unseen['sim_distance_argmin_combined'] = np.log(plagerize_unseen['sim_distance_argmin_combined']+0.0001)

In [131]:
plagerize_unseen = plagerize_unseen.drop(columns=['pair_x', 'song_1', 'song_2', 'pair_y', 'sim_distance', 'sim_distance_argmin'])
plagerize_unseen.reset_index(drop=True, inplace=True)
plagerize_unseen = h2o.H2OFrame(plagerize_unseen)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [132]:
pred_unseen = aml.predict(plagerize_unseen)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [133]:
pred_unseen

predict,p0,p1
0,0.645903,0.354097
0,0.891783,0.108217
1,0.612703,0.387297
1,0.565323,0.434677
1,0.404694,0.595306
0,0.62433,0.37567
0,0.834967,0.165033
0,0.756806,0.243194
1,0.14575,0.85425
1,0.157077,0.842923


In [134]:
results_unseen = pd.concat([plagerize_unseen.as_data_frame(), pred_unseen.as_data_frame()], axis=1)
results_unseen['rank'] = results_unseen['p1'].rank(ascending=False)

results_unseen = results_unseen.sort_values(by=['rank'], ascending=True)
results_unseen

,sim_distance_bass,sim_distance_drums,sim_distance_vocals,sim_distance_other,sim_distance_argmin_combined,predict,p0,p1,rank
13,-3.377752,-3.296034,-3.686204,-3.564177,-3.711500,1,0.115353,0.884647,1.0
8,-3.176397,-3.252703,-2.836602,-3.490970,-3.490970,1,0.145750,0.854250,2.0
9,-3.880038,-2.531039,-2.979819,-3.406609,-3.880038,1,0.157077,0.842923,3.0
4,-1.731434,-3.445135,-3.127537,-3.144894,-3.445135,1,0.404694,0.595306,4.0
15,-3.130439,-3.375734,-2.162241,-3.175864,-3.529704,1,0.414288,0.585712,5.0
14,-3.180531,-3.027164,-2.616983,-3.170255,-3.276949,1,0.441112,0.558888,6.0
20,-2.514196,-3.412679,-2.862315,-3.117153,-3.412679,1,0.536015,0.463985,7.0
25,-3.110523,-3.140508,-3.007707,-3.255698,-3.287768,1,0.563437,0.436563,8.0
3,-2.921536,-3.036883,-2.442322,-3.098474,-3.098474,1,0.565323,0.434677,9.0
2,-3.021577,-3.274226,-3.318774,-2.753680,-3.318774,1,0.612703,0.387297,10.0


In [157]:
aml2 = H2OAutoML(max_models=100, seed=1, balance_classes = True, exclude_algos = ["GBM"], stopping_metric='logloss')
aml2.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [158]:
# View the AutoML Leaderboard
lb2 = aml2.leaderboard
lb2.head(rows=lb2.nrows) 

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
DeepLearning_grid__1_AutoML_20200618_010541_model_6,0.905902,1.47577,0.853138,0.173189,0.47103,0.22187
DeepLearning_grid__3_AutoML_20200618_010541_model_2,0.902356,1.10437,0.801165,0.153179,0.501927,0.251931
DeepLearning_grid__3_AutoML_20200618_010541_model_7,0.898556,0.763057,0.836082,0.183827,0.327623,0.107337
DeepLearning_grid__2_AutoML_20200618_010541_model_7,0.897416,0.319443,0.809103,0.16806,0.311305,0.0969107
DeepLearning_grid__3_AutoML_20200618_010541_model_1,0.89615,2.63214,0.786621,0.185094,0.664917,0.442114
DeepLearning_grid__2_AutoML_20200618_010541_model_5,0.895517,1.29603,0.796332,0.168503,0.465807,0.216976
DeepLearning_grid__3_AutoML_20200618_010541_model_10,0.892857,1.40145,0.803036,0.160841,0.445919,0.198844
DeepLearning_grid__2_AutoML_20200618_010541_model_8,0.890198,1.03643,0.824637,0.173189,0.459556,0.211191
DeepLearning_grid__1_AutoML_20200618_010541_model_5,0.883739,1.88994,0.782815,0.163817,0.571476,0.326584
StackedEnsemble_BestOfFamily_AutoML_20200618_010541,0.883359,0.287005,0.816748,0.154445,0.283025,0.0801029


In [159]:
aml2.leader

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_grid__1_AutoML_20200618_010541_model_6


Status of Neuron Layers: predicting target, 2-class classification, bernoulli distribution, CrossEntropy loss, 4,002 weights/biases, 56.1 KB, 1,323,840 training samples, mini-batch size 1


,,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
0,,1,5,Input,5,,,,,,,,,
1,,2,500,RectifierDropout,30,0,0,0.0948417,0.190474,0,0.130583,0.648805,-0.4535,0.454669
2,,3,2,Softmax,,0,0,0.116843,0.203498,0,-0.287446,1.7959,-13.9339,0.856533




ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.003606874034523682
RMSE: 0.06005725630199636
LogLoss: 0.014922674337819183
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9649643070089776: 


,,0,1,Error,Rate
0,0,168.0,0.0,0.0,(0.0/168.0)
1,1,0.0,168.0,0.0,(0.0/168.0)
2,Total,168.0,168.0,0.0,(0.0/336.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.9209302,0.072802216,1.0,0.88372093,0.81395346,0.95348835,0.95348835
1,auc,0.92717767,0.08168107,1.0,0.8090909,0.875817,0.9836601,0.96732026
2,aucpr,0.86196584,0.13269758,1.0,0.72241664,0.71788657,0.95133847,0.9181876
3,err,0.07906977,0.072802216,0.0,0.11627907,0.18604651,0.046511628,0.046511628
4,err_count,3.4,3.130495,0.0,5.0,8.0,2.0,2.0
5,f0point5,0.8293923,0.15820979,1.0,0.7894737,0.5797101,0.8888889,0.8888889
6,f1,0.83006537,0.13956966,1.0,0.7058824,0.6666667,0.8888889,0.8888889
7,f2,0.8400779,0.13616619,1.0,0.63829786,0.78431374,0.8888889,0.8888889
8,lift_top_group,4.5866666,0.26168966,4.3,4.3,4.7777777,4.7777777,4.7777777
9,logloss,1.4757744,0.77236855,1.1861598,2.56315,1.8991042,1.1776481,0.5528102



See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2020-06-18 01:09:47,0.000 sec,None,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2020-06-18 01:09:47,3 min 52.485 sec,45405 obs/sec,10.0,1,3360.0,0.683058,4.984687,-0.866272,0.890660,0.881439,2.0,0.226190
2,,2020-06-18 01:09:52,3 min 57.509 sec,100813 obs/sec,1520.0,152,510720.0,0.300829,0.385952,0.638007,0.997732,0.997631,2.0,0.017857
3,,2020-06-18 01:09:57,4 min 2.518 sec,116104 obs/sec,3470.0,347,1165920.0,0.177339,0.096239,0.874204,0.999858,0.999857,2.0,0.002976
4,,2020-06-18 01:09:58,4 min 3.700 sec,118422 obs/sec,3940.0,394,1323840.0,0.060057,0.014923,0.985573,1.000000,1.000000,2.0,0.000000



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,sim_distance_other,1.000000,1.000000,0.278376
1,sim_distance_argmin_combined,0.829031,0.829031,0.230782
2,sim_distance_vocals,0.754230,0.754230,0.209959
3,sim_distance_bass,0.532916,0.532916,0.148351
4,sim_distance_drums,0.476091,0.476091,0.132532


In [160]:
pred2 = aml2.predict(test)
pred2

deeplearning prediction progress: |███████████████████████████████████████| 100%


predict,p0,p1
1,3.38781e-50,1
1,4.89301e-109,1
1,1.65899e-37,1
1,0.0059793,0.994021
0,1,1.10701e-63
0,1,1.13775e-34
1,5.88561e-172,1
0,0.99669,0.00330997
0,0.999994,6.28546e-06
0,1,2.36223e-19


In [161]:
results2 = pd.concat([test.as_data_frame(), pred2.as_data_frame()], axis=1)
results2['rank'] = results2['p1'].rank(ascending=False)

In [162]:
results2_sorted = results2.sort_values(by=['rank'], ascending=True)
results2_sorted

,sim_distance_bass,sim_distance_drums,sim_distance_vocals,sim_distance_other,target,sim_distance_argmin_combined,predict,p0,p1,rank
0,-3.396255,-3.318984,-3.726317,-3.307305,1,-3.752329,1,3.387806e-50,1.000000e+00,3.5
11,-2.226876,-3.649815,-3.593604,-3.713786,1,-3.862291,1,8.853730e-65,1.000000e+00,3.5
6,-3.748001,-3.524138,-4.008976,-4.432945,1,-4.432945,1,5.885614e-172,1.000000e+00,3.5
14,-3.601015,-3.368839,-3.177789,-3.862225,1,-4.050557,1,1.759876e-74,1.000000e+00,3.5
2,-3.217733,-2.925328,-2.535029,-3.505658,1,-3.812027,1,1.658990e-37,1.000000e+00,3.5
1,-3.793328,-3.451294,-3.797983,-3.908221,1,-4.110113,1,4.893007e-109,1.000000e+00,3.5
25,-2.437577,-2.955856,-2.886427,-3.238617,0,-3.365747,1,9.909716e-05,9.999009e-01,7.0
15,-2.782669,-3.015499,-2.328491,-2.815056,0,-3.015499,1,4.013327e-04,9.995987e-01,8.0
37,-2.994229,-3.466758,-3.234972,-3.114258,0,-3.466758,1,2.885612e-03,9.971144e-01,9.0
3,-2.739924,-3.098317,-3.356697,-3.304992,1,-3.356697,1,5.979300e-03,9.940207e-01,10.0


In [163]:
#find Average Precision
tp = results2_sorted['target'].values
conf = results2_sorted['p1'].values
pred_cls = results2_sorted['predict'].values
target_cls = [results2_sorted['target'].values]

p, r, ap, f1, unique_classes = ap_per_class(tp, conf, pred_cls, target_cls)

Computing AP: 100%|██████████| 2/2 [00:00<00:00, 1278.36it/s]


In [164]:
#MAP
np.mean(ap)

0.2719764142287931

In [165]:
perf2 = aml2.leader.model_performance(valid)
perf2


ModelMetricsBinomial: deeplearning
** Reported on test data. **

MSE: 0.27584510106617555
RMSE: 0.5252095782315623
LogLoss: 3.128556380625003
Mean Per-Class Error: 0.22857142857142865
AUC: 0.7904761904761906
AUCPR: 0.6650018748993816
Gini: 0.5809523809523811

Confusion Matrix (Act/Pred) for max f1 @ threshold = 4.00081380260415e-08: 


,,0,1,Error,Rate
0,0,19.0,16.0,0.4571,(16.0/35.0)
1,1,0.0,15.0,0.0,(0.0/15.0)
2,Total,19.0,31.0,0.32,(16.0/50.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,4.000814e-08,0.652174,26.0
1,max f2,4.000814e-08,0.824176,26.0
2,max f0point5,1.000000e+00,0.714286,0.0
3,max accuracy,1.000000e+00,0.800000,0.0
4,max precision,1.000000e+00,1.000000,0.0
5,max recall,4.000814e-08,1.000000,26.0
6,max specificity,1.000000e+00,1.000000,0.0
7,max absolute_mcc,4.000814e-08,0.512516,26.0
8,max min_per_class_accuracy,2.627828e-03,0.657143,17.0
9,max mean_per_class_accuracy,4.000814e-08,0.771429,26.0



Gains/Lift Table: Avg response rate: 30.00 %, avg score: 30.66 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.10,1.000000e+00,3.333333,3.333333,1.0,1.000000e+00,1.000000,1.000000,0.333333,0.333333,233.333333,233.333333
1,,2,0.10,1.000000e+00,0.000000,3.333333,0.0,0.000000e+00,1.000000,1.000000,0.000000,0.333333,-100.000000,233.333333
2,,3,0.16,9.999998e-01,0.000000,2.083333,0.0,9.999999e-01,0.625000,1.000000,0.000000,0.333333,-100.000000,108.333333
3,,4,0.20,9.999960e-01,1.666667,2.000000,0.5,9.999982e-01,0.600000,1.000000,0.066667,0.400000,66.666667,100.000000
4,,5,0.30,6.023977e-01,1.333333,1.777778,0.4,9.499240e-01,0.533333,0.983308,0.133333,0.533333,33.333333,77.777778
5,,6,0.40,5.393055e-03,0.000000,1.333333,0.0,1.142841e-01,0.400000,0.766052,0.000000,0.533333,-100.000000,33.333333
6,,7,0.50,7.557687e-06,2.666667,1.600000,0.8,2.187024e-03,0.480000,0.613279,0.266667,0.800000,166.666667,60.000000
7,,8,0.60,7.764053e-07,1.333333,1.555556,0.4,4.066950e-06,0.466667,0.511066,0.133333,0.933333,33.333333,55.555556
8,,9,0.70,2.811431e-14,0.666667,1.428571,0.2,1.248289e-08,0.428571,0.438057,0.066667,1.000000,-33.333333,42.857143
9,,10,0.80,2.812880e-31,0.000000,1.250000,0.0,3.724589e-22,0.375000,0.383300,0.000000,1.000000,-100.000000,25.000000


In [ ]:
h2o.cluster().shutdown()